## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP, specs={"no_sdk":True})]
              , connection_list=[False])

connection command:
indy0: False


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [4]:
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,np.pi))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning pipeline

In [5]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ihhwang/.local/lib/python2.7/site-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/ihhwang/.local/lib/python2.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/ihhwang/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/ihhwang/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/ihhwang/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 848, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/home/ihhwang/.lo

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [6]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [8]:
gscene.set_workspace_boundary( -1.5, 1.5, -1, 1, -0.1, 1.75)

## add indy tool

In [9]:
gscene = gscene
robot_name="indy0"
face_name="brush_face"

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="{}_adapter".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(0, 0, 0.0025), dims=(0.09, 0.09, 0.005), rpy=(0, 0, 0), color=(0.8, 0.8, 0.8, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="{}_adapter_col".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(0, 0, 0.0025), dims=(0.13, 0.13, 0.005), rpy=(0, 0, 0), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_hindge0".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(0, 0, 0.0125), dims=(0.022, 0.036, 0.025), rpy=(0, 0, 0), color=(0.8, 0.8, 0.8, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_hindge0_col".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(0, 0, 0.0125), dims=(0.062, 0.076, 0.025), rpy=(0, 0, 0), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_bar".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.053, 0, 0.068), dims=(0.011, 0.020, 0.15), rpy=(0, 3*pi/4, 0), color=(0.8, 0.8, 0.8, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_bar_col".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.053, 0, 0.068), dims=(0.051, 0.060, 0.15), rpy=(0, 3*pi/4, 0), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_hindge1".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.1085,0,0.121), dims=(0.025, 0.036, 0.022), rpy=(0, 0, 0), color=(0.8, 0.8, 0.8, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_hindge1_col".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.1085,0,0.121), dims=(0.025, 0.076, 0.062), rpy=(0, 0, 0), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="{}_brushbase".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.1285,0,0.121), dims=(0.08, 0.08, 0.015), rpy=(0, pi/2, 0), color=(0.8, 0.8, 0.8, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="{}_brushbase_col".format(robot_name),
                   link_name="{}_tcp".format(robot_name),
                   center=(-0.1285,0,0.121), dims=(0.12, 0.12, 0.015), rpy=(0, pi/2, 0), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name=face_name, link_name="{}_tcp".format(robot_name),
                   center=(-0.147,0,0.121), dims=(0.037, 0.10, 0.34), rpy=(0, 0, pi), color=(1.0, 1.0, 0.94, 1),
                   collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="{}_col".format(face_name), link_name="{}_tcp".format(robot_name),
                   center=(-0.127,0,0.121), dims=(0.057, 0.10, 0.36), rpy=(0, 0, pi), color=(0.0, 0.8, 0.0, 0.5),
                   collision=True, fixed=True)

## recieve table dimension

In [10]:
TABLE_X = 1
TABLE_Y = 1
TABLE_Z = 1
TABLE_R = 0
TABLE_P = 0
TABLE_Y = 0
TABLE_HEIGHT = 0.3
TABLE_DEPTH = 1
TABLE_WIDTH = 1
TOOL_DIM = [0.32, 0.08]

## load dataset

In [12]:
import numpy as np
dataset = np.load('scripts/demo_202107/dataset.npy', allow_pickle=True)
TABLE_HEIGHT = np.round(TABLE_HEIGHT, 3)
len_dat = len(dataset)
for i in range(len_dat):
    print(dataset[i][0])
    if ((TABLE_HEIGHT >= dataset[i][0]) and
        (TABLE_HEIGHT < dataset[i+1][0] if i < len_dat-1 else TABLE_HEIGHT < dataset[i][0]+0.04)):
        data = dataset[i]
        break
        
height_goal, depth_criteria, opt_depth_sort, optimal_list, width_list, area_list = data


# reduce areas by sensor error margin
SENSOR_MARGIN = 0.1
optimal_list = [[pt1, pt2, round(width-SENSOR_MARGIN, 3), round(depth-SENSOR_MARGIN, 3)] for pt1, pt2, width, depth in optimal_list]
optimal_dict = {opt_tem[-1] : opt_tem for opt_tem in optimal_list}
# width_list = [width-width_list for width in width_list]
opt_depth_sort = np.round(np.subtract(opt_depth_sort, SENSOR_MARGIN), 3)

0.15
0.2
0.25
0.3


In [13]:
# select area depth based on table depth
if TABLE_DEPTH <= depth_criteria:
    area_depth_obj = TABLE_DEPTH 
else:
    if TABLE_DEPTH > max(opt_depth_sort) *2:
        print("==========================================================")
        print("=========== TOO DEEP TABLE! CAN'T FINISH WIPING TASK =============")
        print("==========================================================")
        raise(RuntimeError("TOO DEEP TABLE! CAN'T FINISH WIPING TASK"))
    else:
        area_depth_obj = TABLE_DEPTH/2

# select optimal item for the area depth
idx_depth = np.min(np.where(np.subtract(opt_depth_sort, area_depth_obj) > 0)[0])
area_depth = opt_depth_sort[idx_depth]
area_corner1, area_corner2, area_width, _ = optimal_dict[area_depth]

# fit table width to table
if TABLE_WIDTH < area_width:
    area_width = TABLE_WIDTH

In [14]:
# get area parameters
corner_center = [(area_corner1[0] + area_corner2[0]) / 2, (area_corner1[1] + area_corner2[1]) / 2]
corner_center = np.round(corner_center, 5)
area = area_depth * area_width
area = np.round(area, 5)

In [15]:


if area_depth_obj == depth_criteria:
    area_depth = area_depth_obj
else:
    if len(opt_depth_sort) == 1:
        area_depth = opt_depth_sort[0]
    else:
        for j in range(len(opt_depth_sort)-1):
            if (area_depth_obj >= opt_depth_sort[j]) & (area_depth_obj < opt_depth_sort[j+1]):
                area_depth = opt_depth_sort[j]
                break
            else:
                area_depth = opt_depth_sort[len(opt_depth_sort)-1]
                break
#     area_width_obj = TABLE_WIDTH
#     if len(width_list) == 1:
#         area_width = width_list[0]
#     for j in range(len(width_list)-1):
#         if (area_width_obj >= width_list[j]) & (area_width_obj < width_list[j+1]):
#             area_width = width_list[j]
#             break
#     for j in range(len(area_list)):
#         if (area_list[j][2] == area_width) & (area_list[j][3] == area_depth):
#             area_corner1 = area_list[j][0]
#             area_corner2 = area_list[j][1]
            
            
            
#     for j in range(len(width_list)-1):
#         if (area_width_obj >= width_list[j]) & (area_width_obj < width_list[j+1]):
#             area_width = width_list[j+1]
#             break
#     for j in range(len(area_list)):
#         if (area_list[j][2] == area_width) & (area_list[j][3] == area_depth):
#             area_corner1 = area_list[j][0]
#             area_corner2 = area_list[j][1]
#             area_width = area_width_obj

corner_center = [(area_corner1[0] + area_corner2[0]) / 2, (area_corner1[1] + area_corner2[1]) / 2]
corner_center = np.round(corner_center, 5)
area = area_depth * area_width
area = np.round(area, 5)

## decompose wiping area

In [16]:
if area_depth < TABLE_DEPTH:
    num_depth = 2
else:
    num_depth = 1

num_width, rem = divmod(TABLE_WIDTH, area_width)
if not rem == 0:
    num_width += 1
num_width = int(num_width)

num_area = num_width * num_depth

## add sweep face

In [17]:
from pkg.geometry.geotype import GEOTYPE

In [18]:
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (area_depth,area_width,0.01), (corner_center[0],corner_center[1],height_goal), rpy=(0,0,0), 
                           color=(0.8,0.8,0.8,0.8), display=True, fixed=True, collision=True)

In [19]:
track_face = gscene.copy_from(track, new_name="track_face", collision=False, color=(0.8,0.8,0.8,0.2))
TRACK_DIM = np.copy(track_face.dims)
track_face.dims = (3, 3, track.dims[2])

In [20]:
gscene.update_markers_all()

# coverage planning

In [21]:
from pkg.planning.task.custom_rules.coverage import *

In [22]:
TOOL_DIM = [0.32, 0.08]
TACK_WIDTH = 0.36
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[0]-TOOL_DIM[0], TOOL_DIM[0])).astype(np.int)+1
TRACK_STEP = TRACK_DIM[0]/TRACK_NUM

In [23]:
img = grayscale("scripts/demo_202107/input.jpg")

table_X = TRACK_DIM[0]
table_Y = TRACK_DIM[1]
table_x = img.shape[0]
table_y = img.shape[1]

path_gap_real = TRACK_STEP
path_gap_img = int(path_gap_real * (table_x / table_X))
wp1_pos_img, wp2_pos_img = make_waypoint(img, path_gap_img)

wp1_pos_real = np.empty([TRACK_NUM,3])
wp1_pos_real[:,0] = wp1_pos_img[:,0] * (table_X / table_x)
wp1_pos_real[:,1] = wp1_pos_img[:,1] * (table_Y / table_y)
wp1_pos_real[:,1] = -wp1_pos_real[:,1];
wp1_pos_real[:,2] = 0

wp2_pos_real = np.empty([TRACK_NUM,3])
wp2_pos_real[:,0] = wp2_pos_img[:,0] * (table_X / table_x)
wp2_pos_real[:,1] = wp2_pos_img[:,1] * (table_Y / table_y)
wp2_pos_real[:,1] = -wp2_pos_real[:,1];
wp2_pos_real[:,2] = 0

In [24]:
wp1_pos_real[:,0] -= TRACK_DIM[0]/2
wp1_pos_real[:,1] +=  TRACK_DIM[1]/2
wp2_pos_real[:,0] -= TRACK_DIM[0]/2
wp2_pos_real[:,1] += TRACK_DIM[1]/2

## add wp

In [25]:
TRC_THIC = TRACK_DIM[2]
track_list = []

wp1_pos_real[:,1] -= TOOL_DIM[1]/2
wp2_pos_real[:,1] += TOOL_DIM[1]/2

for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             wp1_pos_real[i_trc,:], rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
#     wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
#                              (0.4, -0.01, height_goal), rpy=(0,0,0), 
#                              color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             wp2_pos_real[i_trc,:], rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TACK_WIDTH,TRACK_DIM[1],TRC_THIC), 
                             center=(wp1_pos_real[i_trc,:]+wp2_pos_real[i_trc,:])/2,rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [26]:
gscene.update_markers_all()

## add blocking structure on table

In [27]:
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

## Register binders

In [28]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [29]:
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), 
                     rpy=(0,np.pi/2*3,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane, point=None)

## add objects

In [30]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask, AbstractObject
from pkg.planning.constraint.constraint_subject import SweepLineTask
from pkg.planning.constraint.constraint_subject import SweepFrame

In [31]:
for sname in pscene.subject_name_list:
    pscene.remove_subject(sname)

In [32]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [33]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [34]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [35]:
mplan.motion_filters = checkers_all

In [36]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [37]:
HOME_POSE = [-0.        , -0.        ,  np.pi*4/6, -0.        ,  -np.pi/6, -0.        ]

In [38]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
# initial_state = pscene.initialize_state(crob.home_pose)
initial_state = pscene.initialize_state(HOME_POSE)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

(0, 0)


In [39]:
pscene.subject_name_list

['sweep1', 'sweep2']

# Node Sampler

In [40]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [41]:
# from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
# tplan.custom_rule = SweepEntranceControlRule(pscene)

## Make Plan

In [42]:
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()
tplan.prepare()
mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = 0
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=10, multiprocess=True, timeout=1, timeout_constrained=2, add_homing=True, post_optimize=False)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules(False)
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_ori = snode_schedule
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 5, 1)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

(0, 0)
Use 36/36 agents
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(1, 0)
transition motion tried: True
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(1, 0) = success
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 1 - (1, 0)->(2, 0)
branching: 0->1 (0.13/10.0 s, steps/err: 24(57.6319694519 ms)/0.000769737924892)
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
try: 1 - (1, 0)->(2, 0)
result: 0 - (0, 0)->(0, 1) = success
try transition motion
branching: 0->2 (0.14/10.0 s, steps/err: 22(61.7580413818 ms)/0.0020837410548)
branching: 0->3 (0.14/10.0 s, steps/err: 22(51.3479709625 ms)/0.00163779432836)
try: 2 - (0, 1)->(0, 2)
try constrained motion
transition motion tried: True
try: 3 - (0, 1)->(0, 2)
try constrained motion
transition motion tried: True
try constra

try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
transition motion tried: True
branching: 0->20 (0.8/10.0 s, steps/err: 22(48.614025116 ms)/0.00174609561328)
result: 0 - (0, 0)->(1, 0) = success
branching: 0->21 (0.8/10.0 s, steps/err: 24(65.3018951416 ms)/0.00183977436813)
transition motion tried: True
try: 21 - (1, 0)->(2, 0)
result: 0 - (0, 0)->(1, 0) = success
branching: 0->22 (0.81/10.0 s, steps/err: 24(53.3149242401 ms)/0.00198449434374)
try: 22 - (1, 0)->(2, 0)
try: 20 - (0, 1)->(0, 2)
try constrained motion
try constrained motion
try constrained motion
constrained motion tried: True
result: 1 - (1, 0)->(2, 0) = success
branching: 1->23 (0.87/10.0 s, steps/err: 271(610.648870468 ms)/0.00156804815515)
++ adding return motion to acquired answer ++
constrained motion tried: True
result: 10 - (0, 1)->(0, 2) = success
branching: 10->25 (1.06/10.0 s, steps/err: 194(391.893863678 ms)/0.00176889934651)

result: 0 - (2, 0)->(2, 1) = success
result: 0 - (2, 0)->(2, 1) = success
branching: 0->2 (0.12/10.0 s, steps/err: 22(51.8538951874 ms)/0.00157141642783)
try: 2 - (2, 1)->(2, 2)
transition motion tried: True
branching: 0->3 (0.12/10.0 s, steps/err: 22(40.962934494 ms)/0.00154131820544)
result: 0 - (2, 0)->(2, 1) = success
try constrained motion
try constrained motion
try: 3 - (2, 1)->(2, 2)
constrained motion tried: True
try: 4 - (2, 1)->(2, 2)
branching: 0->4 (0.13/10.0 s, steps/err: 22(38.2990837097 ms)/0.00173988261773)
try constrained motion
result: 1 - (1, 0)->(2, 0) = success
try constrained motion
branching: 1->38 (2.15/10.0 s, steps/err: 271(1974.28011894 ms)/0.00187018761363)
try: 1 - (2, 1)->(2, 2)
++ adding return motion to acquired answer ++
constrained motion tried: False
try constrained motion
Motion Plan Failure
result: 2 - (0, 1)->(0, 2) = fail
======================= terminated 8: first answer acquired from other agent ===============================
constrained motion

try: 3 - (2, 1)->(2, 2)
======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
try constrained motion
try constrained motion
constrained motion tried: False
Motion Plan Failure
result: 9 - (1, 0)->(2, 0) = fail
constrained motion tried: False
======================= terminated 20: first answer acquired from other agent ===============================
Motion Plan Failure
result: 4 - (1, 0)->(2, 0) = fail
======================= terminated 14: first answer acquired from other agent ===============================
try: 0 - (2, 0)->(2, 1)
try: 0 - (2, 0)->(2, 1)
constrained motion tried: False
Motion Plan Failure
result: 15 - (1, 0)->(2, 0) = fail
======================= terminated 27: first answer acquired from other agent ===============================
try transition motion
constrained motion tried: False
try transition moti

======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 26: first answer acquired from other agent ===============================
======================= terminated 27: first answer acquired from other agent ===============================
transition motion tried: True
result: 0 - (2, 0)->(2, 1) = success
branching: 0->14 (0.75/10.0 s, steps/err: 22(46.7829704285 ms)/0.00109845865587)
======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 24: first answer acquired from other agent ===============================
======================= terminated 29: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 30: first answer acquired from other agent ====

++ adding return motion to acquired answer ++
======================= terminated 4: first answer acquired ===============================
======================= terminated 11: first answer acquired ===============================
======================= terminated 17: first answer acquired ===============================
constrained motion tried: True
======================= terminated 7: first answer acquired ===============================
result: 8 - (2, 1)->(2, 2) = success
branching: 8->27 (1.55/10.0 s, steps/err: 193(987.186908722 ms)/0.00146628267204)
++ adding return motion to acquired answer ++
constrained motion tried: True
result: 4 - (2, 1)->(2, 2) = success
branching: 4->29 (1.63/10.0 s, steps/err: 193(958.0950737 ms)/0.000543185478364)
++ adding return motion to acquired answer ++
constrained motion tried: True
result: 2 - (2, 1)->(2, 2) = success
branching: 2->31 (1.68/10.0 s, steps/err: 194(1327.21495628 ms)/0.00149564838939)
++ adding return motion to acquired answer 

======================= terminated 15: first answer acquired ===============================
======================= terminated 14: first answer acquired ===============================


In [43]:
section_success = len(snode_schedule_all) == sweep_num
if section_success:
    print("go wipe the surface")
else:
    print("try other area")

go wipe the surface


In [44]:
sweep0 = sweep_list[0]

In [45]:
sweep0.geometry.center

(0.68, 0.0, 0.3)

## refine sweep motions 

In [46]:

    
def get_jacobian(gscene, gtem, Q):
    Q_dict = list2dict(Q, gscene.joint_names)
    Jac = []
    for ij, jname in enumerate(gscene.joint_names):    
        joint = gscene.urdf_content.joint_map[jname]
        Tj = T_xyzrpy((joint.origin.xyz, joint.origin.rpy))
        T_link = get_tf(joint.parent, Q_dict, gscene.urdf_content)
        T_bj = np.matmul(T_link, Tj)
        zi = np.matmul(T_bj[:3,:3], joint.axis)        
        T_p = gtem.get_tf(Q_dict)
        dpi = T_p[:3,3]-T_bj[:3,3]
        zp = np.cross(zi, dpi)
        Ji= np.concatenate([zp, zi])
        Jac.append(Ji)
    Jac = np.array(Jac).transpose()
    return Jac
    
def make_sweep_traj(gscene, mplan, gtem, Traj):
    SINGULARITY_CUT = 0.01
    Qi = Traj[0]
    Qf = Traj[-1]
    len_traj = len(Traj)
    
    Qidict = list2dict(Qi, gscene.joint_names)
    Qfdict = list2dict(Qf, gscene.joint_names)
    Ti = gtem.get_tf(Qidict)
    Tf = gtem.get_tf(Qfdict)
    dPabs = np.linalg.norm(Tf[:3,3]-Ti[:3,3])
    DP = dPabs/len_traj
    DIR = np.concatenate([(Tf[:3,3]-Ti[:3,3])/dPabs, [0]*3])
    Q=Qi
    singularity = False
    Traj_wipe = []
    for _ in range(len_traj):
        Jac = get_jacobian(gscene, gtem, Q)
        if np.min(np.abs(np.real(np.linalg.svd(Jac)[1]))) <= SINGULARITY_CUT:
            singularity = True
            print("singular")
            break
        Jinv = np.linalg.inv(Jac)
        dQ=np.matmul(Jinv, np.multiply(DIR, DP))
        Q = Q+dQ
        Traj_wipe.append(Q)
        dlim = np.subtract(RobotSpecs.get_joint_limits(RobotType.indy7), Q[:, np.newaxis])
        if np.any(dlim[:,0] > 0):
            print("min lim: {}".format(np.where(dlim[:,0] > 0)[0]))
            break
        if np.any(dlim[:,1] < 0):
            print("max lim: {}".format(np.where(dlim[:,1] < 0)[0]))
            break
        if not mplan.validate_trajectory([Q]):
            print("col")
            break
        Tnew = gtem.get_tf(list2dict(Q, gscene.joint_names))
        if np.abs(Ti[0,3]-Tnew[0,3])>0.01:
            print("off")
            break
    Traj_wipe.append(Qf)
    Traj_wipe = np.array(Traj_wipe)
    return Traj_wipe
    

def refine_sweep(pscene, mplan, snode_schedule):
    for snode_pre, snode in zip(snode_schedule[:-1], snode_schedule[1:]):
        breaker = False
        for i_n, (ntem1, ntem2) in enumerate(zip(snode_pre.state.node, snode.state.node)):
            if ntem1 != ntem2 and pscene.subject_type_list[i_n] == SweepLineTask:
                if ntem1==1:
                    gtem = gscene.NAME_DICT[snode.state.binding_state[i_n][-1]]
                    snode.traj = make_sweep_traj(pscene.gscene, mplan, gtem, snode.traj)


In [47]:
for snode_schedule in snode_schedule_all:
    refine_sweep(pscene, mplan, snode_schedule)

In [48]:
for snode_schedule in snode_schedule_all:
    ppline.play_schedule(snode_schedule, period=0.002)

(0, 0)->(0, 0)
(0, 0)->(1, 0)
(1, 0)->(2, 0)
(2, 0)->(2, 0)
(2, 0)->(2, 0)
(2, 0)->(2, 1)
(2, 1)->(2, 2)
(2, 2)->(2, 2)


### Test task motion

In [47]:
Traj = ppline.tplan.snode_dict[15].traj
gscene.show_motion(Traj)

In [92]:
gtem = gscene.NAME_DICT["brush_face"]
SINGULARITY_CUT = 0.01
Q0 = np.copy(Traj[-1])
Q0dict = list2dict(Q0, gscene.joint_names)
T0 = gtem.get_tf(Q0dict)
DP = 0.01
DIR = [0,1,0,0,0,0]
Q=Q0
singularity = False
Traj_left = []
for _ in range(1000):
    Jac = get_jacobian(gscene, gtem, Q)
    if np.min(np.abs(np.real(np.linalg.svd(Jac)[1]))) <= SINGULARITY_CUT:
        singularity = True
        print("singular")
        break
    Jinv = np.linalg.inv(Jac)
    dQ=np.matmul(Jinv, np.multiply(DIR, DP))
    Q = Q+dQ
    Traj_left.append(Q)
    dlim = np.subtract(RobotSpecs.get_joint_limits(RobotType.indy7), Q[:, np.newaxis])
    if np.any(dlim[:,0] > 0):
        print("min lim: {}".format(np.where(dlim[:,0] > 0)[0]))
        break
    if np.any(dlim[:,1] < 0):
        print("max lim: {}".format(np.where(dlim[:,1] < 0)[0]))
        break
#     if not mplan.validate_trajectory([Q]):
#         print("col")
#         break
#     Tnew = gtem.get_tf(list2dict(Q, gscene.joint_names))
#     if np.abs(T0[0,3]-Tnew[0,3])>0.01:
#         print("off")
#         break

singular


In [56]:
gscene.show_motion(np.array(Traj_left)[::1, :])

In [51]:
track.center[0], track.center[1]

(0.44, 0.005)

In [ ]:
print(gtimer)

In [ ]:
# for i_ss, snode_schedule in enumerate(snode_schedule_all):
#     for i_s, snode in enumerate(snode_schedule):
#         if snode.traj is not None:
#             save_json("data/traj_{}_{}.json".format(i_ss, i_s), snode.traj[:,:6])

### play schedule

In [ ]:
for snode_schedule in snode_schedule_all:
    ppline.play_schedule(snode_schedule, period=0.001)